In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

/Users/oanaalexandrablanc/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/oanaalexandrablanc/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
# Specify the file path
file_path_test = "unlabelled_test_data.csv"
file_path_training = "training_data.csv"

# Read the CSV file
test_data = pd.read_csv(file_path_test)
training_data = pd.read_csv(file_path_training)

test_data.head()

,id,sentence
0,0,Nous dûmes nous excuser des propos que nous eû...
1,1,Vous ne pouvez pas savoir le plaisir que j'ai ...
2,2,"Et, paradoxalement, boire froid n'est pas la b..."
3,3,"Ce n'est pas étonnant, car c'est une saison my..."
4,4,"Le corps de Golo lui-même, d'une essence aussi..."


In [3]:
# Distribution of difficulty levels
difficulty_distribution = training_data['difficulty'].value_counts()
# Sample sentences
sample_sentences = training_data['sentence'].sample(5, random_state=42)

difficulty_distribution, sample_sentences

(difficulty
 A1    813
 C2    807
 C1    798
 B1    795
 A2    795
 B2    792
 Name: count, dtype: int64,
 596                               Je peux m'asseoir ici ?
 3370    C'est la couleur de nombreux fruits et légumes...
 3048    Pas au point qu'il faille en limiter la consom...
 2908    Les Français ne cèdent pas au chacun pour soi,...
 8       J'ai retrouvé le plaisir de manger un oeuf à l...
 Name: sentence, dtype: object)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create an instance of TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)

# Fit and transform the cleaned text data
X = vectorizer.fit_transform(training_data['sentence'])
y = training_data['difficulty']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape

((3840, 5000), (960, 5000))

In [28]:
y_train = pd.to_numeric(y_train, errors='coerce')

In [29]:
y_train.fillna(y_train.median(), inplace=True)

/Users/oanaalexandrablanc/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1216: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [30]:
X_train.dtype, y_train.dtype

(dtype('float64'), dtype('float64'))